In [167]:
#Import Dependencies
import os
import pandas as pd

In [168]:
#Define pathways for datafiles
path_schools = os.path.join("Resources", "schools_complete.csv")
path_students = os.path.join("Resources", "students_complete.csv")

#Load in datafiles
schools_df = pd.read_csv(path_schools)
students_df = pd.read_csv(path_students)

#Merge data files bases on school
school_data_complete = pd.merge(students_df, schools_df, how="left", on=["school_name", "school_name"])


## Store Basic info for District

In [169]:
#Create snapshot of the district's key metrics
district_num_schools = len(school_data_complete["school_name"].unique())    #Store number of schools
district_num_students = len(school_data_complete["Student ID"].unique())  #Store number of students

district_by_school = school_data_complete.drop_duplicates(subset = ["school_name"]) #Made a DataFrame where there are not duplicates of schools (one student shown for each school)
district_total_budget = district_by_school["budget"].sum()  #Store total budget for district

# Distict Summary Stats

In [170]:
#Average scores
district_math_avg = school_data_complete["math_score"].mean()
district_reading_avg = school_data_complete["reading_score"].mean()

bins = [0, 70, 100] #Bins for failing (below 70) versus passing with max of 100
labels = ["fail", "pass"]   #Two bins: fail or pass
school_data_complete["pass/fail_reading"] = pd.cut(school_data_complete["reading_score"], bins, labels = labels, include_lowest=True) #Make new column for pass or failing reading
school_data_complete["pass/fail_math"] = pd.cut(school_data_complete["math_score"], bins, labels = labels, include_lowest=True)     #Make new column for pass or failing math

district_math_count = school_data_complete.groupby(["pass/fail_math"]).count()["Student ID"]    #Make series of students who fail and pass math
district_math_perc = district_math_count[1]/(district_math_count.sum())*100     #Calculate passing rate for math

district_reading_count = school_data_complete.groupby(["pass/fail_reading"]).count()["Student ID"]    #Make series of students who fail and pass reading
district_reading_perc = district_reading_count[1]/(district_reading_count.sum())*100    #Calculate passing rate for reading

#Calculate how mamy students passed reading AND math
district_pass_both = school_data_complete.loc[(school_data_complete["pass/fail_math"] == "pass") & (school_data_complete["pass/fail_reading"] == "pass"), ["student_name", "pass/fail_math", "pass/fail_reading"]]
district_pass_both_count = district_pass_both["student_name"].count()
district_pass_both_perc = district_pass_both_count/(school_data_complete["student_name"].count())*100


In [172]:

#Create a Data Frame to Hold District Information
district_summary_df = pd.DataFrame({
    "Total Schools": [district_num_schools],
    "Total Students": [district_num_students],
    "Total Budget": [district_total_budget],
    "Average Math Score": [district_math_avg],
    "Average Reading Score": [district_reading_avg],
    "% Passing Math": [district_math_perc],
    "% Passing Reading": [district_reading_perc],
    "% Overall Passing": [district_pass_both_perc]
})

#Clean up formatting
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:,.2f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:,.2f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:,.2f}%".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:,.2f}%".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:,.2f}%".format)

pd.DataFrame(district_summary_df)

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",78.99,81.88,72.39%,82.97%,60.80%


# School's Summaries

In [173]:
#School Basic info

gby_schools = school_data_complete.groupby(["school_name"]) #Create Groupby object organized by the school

district_by_school_sorted = district_by_school.sort_values("school_name")

school_names = (district_by_school_sorted["school_name"]).values.tolist() #Make a list of school names
school_types = (district_by_school_sorted["type"]).values.tolist()   #make a list of school types that have the same indicies as the lit of school names
school_num_students = (gby_schools["student_name"].count()).values.tolist() #Make a list of the total the number of students for each school
school_budgets = (district_by_school_sorted["budget"]).values.tolist()     #make a list of budgets that have the same indicies as the list of schoosl


In [174]:
#Calculations for per student budget
school_budget_per_student = []
for i in range(district_num_schools):  #Loop through the number of schools
    school_budget_per_student.append((school_budgets[i])/(school_num_students[i]))  #Calculate the budget per student and add to list 


In [175]:
#Calculations for scores 
avg_math_scores = (gby_schools["math_score"].mean()).values.tolist()
avg_read_scores = (gby_schools["reading_score"].mean()).values.tolist()

#############################################################################

#Create df for kids who only passed math. Include columns for school_name and Student ID
passing_math_df = school_data_complete.loc[school_data_complete["pass/fail_math"] == "pass", ["school_name", "Student ID"]]
schools_passing_math_counts = (passing_math_df.groupby("school_name")["Student ID"].count()).values.tolist()    #Groupby school

#Loop through list of school and make a list with the percet of students who passed math
schools_pass_math_perc = []
for i in range(district_num_schools):
    schools_pass_math_perc.append((schools_passing_math_counts[i]/school_num_students[i])*100)

#############################################################################

#Create df for kids who only passed reading. Include columns for school_name and Student ID
passing_reading_df = school_data_complete.loc[school_data_complete["pass/fail_reading"] == "pass", ["school_name", "Student ID"]]
schools_passing_reading_counts = (passing_reading_df.groupby("school_name")["Student ID"].count()).values.tolist()

#Loop through list of school and make a list with the percet of students who passed reading
schools_pass_reading_perc = []
for i in range(district_num_schools):
    schools_pass_reading_perc.append((schools_passing_reading_counts[i]/school_num_students[i])*100)

In [176]:
#Percent of kids who passed both reading AND math at each school
#Create df of kids who passed both reading AND math
schools_pass_df = school_data_complete.loc[(school_data_complete["pass/fail_math"] == "pass") & (school_data_complete["pass/fail_reading"] == "pass"), ["school_name", "Student ID"]]
schools_pass_counts = schools_pass_df.groupby("school_name")["Student ID"].count()  #Groupby school and count number of ids

#Loop through schools and calculate percent at each school
schools_pass_both = []
for i in range(district_num_schools):
    schools_pass_both.append((schools_pass_counts[i])/(school_num_students[i])*100)



In [197]:
#Create Data Frame with Summaries for each school.

school_summary_df = pd.DataFrame({
    "School Name": school_names,
    "School Type": school_types,
    "Total Students": school_num_students,
    "Total Budget": school_budgets,
    "Per Student Budget": school_budget_per_student,
    "Average Math Score": avg_math_scores,
    "Average Reading Score": avg_read_scores,
    "% Passing Math": schools_pass_math_perc,
    "% Passing Reading": schools_pass_reading_perc,
    "% Overall Passing": schools_pass_both
})

#Make an original copy to use later (before formatting):
school_summary_og_df = pd.DataFrame({
    "School Name": school_names,
    "School Type": school_types,
    "Total Students": school_num_students,
    "Total Budget": school_budgets,
    "Per Student Budget": school_budget_per_student,
    "Average Math Score": avg_math_scores,
    "Average Reading Score": avg_read_scores,
    "% Passing Math": schools_pass_math_perc,
    "% Passing Reading": schools_pass_reading_perc,
    "% Overall Passing": schools_pass_both
})

#Clean Up Formatting 
school_summary_df["Total Students"] = school_summary_df["Total Students"].map("{:,}".format)
school_summary_df["Total Budget"] = school_summary_df["Total Budget"].map("${:,}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].map("${:,}".format)
school_summary_df["Average Math Score"] = school_summary_df["Average Math Score"].map("{:,.2f}".format)
school_summary_df["Average Reading Score"] = school_summary_df["Average Reading Score"].map("{:,.2f}".format)
school_summary_df["% Passing Math"] = school_summary_df["% Passing Math"].map("{:,.2f}%".format)
school_summary_df["% Passing Reading"] = school_summary_df["% Passing Reading"].map("{:,.2f}%".format)
school_summary_df["% Overall Passing"] = school_summary_df["% Overall Passing"].map("{:,.2f}%".format)

school_summary_og_df

,School Name,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,64.630225,79.300643,51.145498
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,89.558665,93.864370,84.015070
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,63.750424,78.433367,49.915226
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,65.753925,77.510040,51.296093
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,89.713896,93.392371,83.651226
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,64.746494,78.187702,50.161812
6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,90.632319,92.740047,84.074941
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,63.318478,78.813850,49.914296
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,63.852132,78.281874,49.800462
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,91.683992,92.203742,84.823285


# Highest Performing Schools by % Overall Passing

In [178]:
#Sort school summary df by % overall passing
highest_performing_df = school_summary_df.sort_values("% Overall Passing", ascending= False)

#Cut the highest performing df to the top 5
highest_performing_df = highest_performing_df.iloc[:5, :]

highest_performing_df

,School Name,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
13,Wilson High School,Charter,"2,283","$1,319,574",$578.0,83.27,83.99,90.93%,93.25%,84.89%
9,Pena High School,Charter,962,"$585,858",$609.0,83.84,84.04,91.68%,92.20%,84.82%
14,Wright High School,Charter,"1,800","$1,049,400",$583.0,83.68,83.95,90.28%,93.44%,84.44%
12,Thomas High School,Charter,"1,635","$1,043,130",$638.0,83.42,83.85,90.21%,92.91%,84.28%
6,Holden High School,Charter,427,"$248,087",$581.0,83.80,83.81,90.63%,92.74%,84.07%


# Lowest Performing Schools by % Overall Passing

In [179]:
#Sort school summary df by % overall passing
lowest_performing_df = school_summary_df.sort_values("% Overall Passing")

#Cut the lowest performing df to the top 5
lowest_performing_df = lowest_performing_df.iloc[:5, :]

lowest_performing_df

,School Name,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
10,Rodriguez High School,District,"3,999","$2,547,363",$637.0,76.84,80.74,64.07%,77.74%,49.44%
8,Johnson High School,District,"4,761","$3,094,650",$650.0,77.07,80.97,63.85%,78.28%,49.80%
7,Huang High School,District,"2,917","$1,910,635",$655.0,76.63,81.18,63.32%,78.81%,49.91%
2,Figueroa High School,District,"2,949","$1,884,411",$639.0,76.71,81.16,63.75%,78.43%,49.92%
5,Hernandez High School,District,"4,635","$3,022,020",$652.0,77.29,80.93,64.75%,78.19%,50.16%


# Math Scores by Grade

In [180]:
math_df = school_data_complete.groupby(["school_name", "grade"])[["math_score"]].mean()

#Clean up formatting
math_df["math_score"] = math_df["math_score"].map("{:,.2f}".format)

mathscoresbygrade_df = pd.pivot_table(math_df, index='school_name', columns='grade',
                        aggfunc='mean', fill_value=0)
mathscoresbygrade_df


math_score                     
grade                       10th   11th   12th    9th
school_name                                          
Bailey High School         77.00  77.52  76.49  77.08
Cabrera High School        83.15  82.77  83.28  83.09
Figueroa High School       76.54  76.88  77.15  76.40
Ford High School           77.67  76.92  76.18  77.36
Griffin High School        84.23  83.84  83.36  82.04
Hernandez High School      77.34  77.14  77.19  77.44
Holden High School         83.43  85.00  82.86  83.79
Huang High School          75.91  76.45  77.23  77.03
Johnson High School        76.69  77.49  76.86  77.19
Pena High School           83.37  84.33  84.12  83.63
Rodriguez High School      76.61  76.40  77.69  76.86
Shelton High School        82.92  83.38  83.78  83.42
Thomas High School         83.09  83.50  83.50  83.59
Wilson High School         83.72  83.20  83.04  83.09
Wright High School         84.01  83.84  83.64  83.26

# Reading Scores by Grade

In [181]:
reading_df = school_data_complete.groupby(["school_name", "grade"])[["reading_score"]].mean()

#Clean up formatting
reading_df["reading_score"] = reading_df["reading_score"].map("{:,.2f}".format)

readingscoresbygrade_df = pd.pivot_table(reading_df, index='school_name', columns='grade',
                        aggfunc='mean', fill_value=0)

columnsTitles = ["9th", "10th", "11th", "12th"]

#readingscoresbygrade_f_df = readingscoresbygrade_df.reindex(columns=columnsTitles)




               grade
reading_score  10th     float64
               11th     float64
               12th     float64
               9th      float64
dtype: object

# Scores by Schools spending    

In [196]:
bins_spending = [0, 585, 630, 645, 680 ] #Bins with $20 range
labels_spending = ["<585", "585-630", "630-645", "645-680"]   #Four bins 
spending_classifications = pd.cut(school_summary_og_df["Per Student Budget"], bins_spending, labels = labels_spending, include_lowest=True) #Make list

school_summary_spending_df = school_summary_og_df
school_summary_spending_df["Spending Classifications"] = spending_classifications
spending_df = school_summary_spending_df.groupby("Spending Classifications")[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]].mean()

#Formatting
spending_df["Average Math Score"] = spending_df["Average Math Score"].map("{:,.2f}".format)
spending_df["Average Reading Score"] = spending_df["Average Reading Score"].map("{:,.2f}".format)
spending_df["% Passing Math"] = spending_df["% Passing Math"].map("{:,.2f}%".format)
spending_df["% Passing Reading"] = spending_df["% Passing Reading"].map("{:,.2f}%".format)
spending_df["% Overall Passing"] = spending_df["% Overall Passing"].map("{:,.2f}%".format)


spending_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Classifications,,,,,
<585,83.46,83.93,90.35%,93.33%,84.36%
585-630,81.90,83.16,83.98%,89.38%,75.70%
630-645,78.52,81.62,70.95%,81.65%,58.73%
645-680,77.00,81.03,63.97%,78.43%,49.96%


# Scores by School Size

In [200]:
bins_size = [0, 1000, 2000, 5000] #Bins for school size (pop)
labels_size = ["Small (<1000)", "Medium (1000-2000", "Large (2000-3000"]   #Three bins 
school_size_classifications = pd.cut(school_summary_og_df["Total Students"], bins_size, labels = labels_size, include_lowest=True) #Make list

school_summary_size_df = school_summary_og_df
school_summary_size_df["Scores by School Size"] = school_size_classifications
scorebysize_df = school_summary_size_df.groupby("Scores by School Size")[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]].mean()

#Formatting
scorebysize_df["Average Math Score"] = scorebysize_df["Average Math Score"].map("{:,.2f}".format)
scorebysize_df["Average Reading Score"] = scorebysize_df["Average Reading Score"].map("{:,.2f}".format)
scorebysize_df["% Passing Math"] = scorebysize_df["% Passing Math"].map("{:,.2f}%".format)
scorebysize_df["% Passing Reading"] = scorebysize_df["% Passing Reading"].map("{:,.2f}%".format)
scorebysize_df["% Overall Passing"] = scorebysize_df["% Overall Passing"].map("{:,.2f}%".format)

scorebysize_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Scores by School Size,,,,,
Small (<1000),83.82,83.93,91.16%,92.47%,84.45%
Medium (1000-2000,83.37,83.86,89.93%,93.24%,83.92%
Large (2000-3000,77.75,81.34,67.63%,80.19%,54.57%


# Scores by School Type 

In [202]:
scoresbytype_df = school_summary_og_df.groupby("School Type")[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]].mean()

#Formatting
scoresbytype_df["Average Math Score"] = scoresbytype_df["Average Math Score"].map("{:,.2f}".format)
scoresbytype_df["Average Reading Score"] = scoresbytype_df["Average Reading Score"].map("{:,.2f}".format)
scoresbytype_df["% Passing Math"] = scoresbytype_df["% Passing Math"].map("{:,.2f}%".format)
scoresbytype_df["% Passing Reading"] = scoresbytype_df["% Passing Reading"].map("{:,.2f}%".format)
scoresbytype_df["% Overall Passing"] = scoresbytype_df["% Overall Passing"].map("{:,.2f}%".format)

scoresbytype_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,90.36%,93.05%,84.17%
District,76.96,80.97,64.30%,78.32%,50.24%
